In [25]:
#27/8/24 misc dataset exploration
    #trying to find correlation between dataset size and model output
import gc
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '999'
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import nnmetrics
from Function_Repo import setbalance
import math
import tensorflow as tf
from tensorflow import keras
from keras.layers import BatchNormalization
import matplotlib.pyplot as plt

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/ProcessedRaws/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

In [26]:
lengths = []
for item in datasets:
    lengths.append(len(item['Toxicity_Value'].values))

In [27]:
lengths

[688,
 480,
 2889,
 2119,
 1348,
 1674,
 847,
 41,
 1836,
 760,
 2106,
 516,
 365,
 1021,
 726,
 1547,
 1547,
 604,
 69,
 7485,
 625,
 1547]

In [28]:
loops = 0
for item in namelist:
    print(item, ':', lengths[loops], loops)
    loops += 1

Cardiotoxicity-30.csv : 688 0
NR-AR-LBD.csv : 480 1
Hepatotoxicity.csv : 2889 2
SR-ARE.csv : 2119 3
Respiratory_Toxicity.csv : 1348 4
NR-AhR.csv : 1674 5
SR-p53.csv : 847 6
Reproductive_Toxicity.csv : 41 7
NR-ER.csv : 1836 8
SR-HSE.csv : 760 9
SR-MMP.csv : 2106 10
SR-ATAD5.csv : 516 11
NR-PPAR-gamma.csv : 365 12
Carcinogenicity.csv : 1021 13
NR-ER-LBD.csv : 726 14
Cardiotoxicity-10.csv : 1547 15
Cardiotoxicity-5.csv : 1547 16
NR-aromatase.csv : 604 17
Developmental_Toxicity.csv : 69 18
Ames_Mutagenicity.csv : 7485 19
NR-AR.csv : 625 20
Cardiotoxicity-1.csv : 1547 21


In [29]:
#NR-AR-LBD_Chemopy.csv is the set we will be using for testing due to its relatively smalle size and mcc
datasets[1]

,SMILES,Toxicity_Value
0,CC(C)C[C@H](NC(=O)[C@H](Cc1ccccc1)NC(=O)c1cncc...,0
26,O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1...,0
52,Clc1ccc(C(Cn2ccnc2)OCc2csc3c(Cl)cccc23)c(Cl)c1,0
78,Nc1cc(-c2ccncc2)c[nH]c1=O,0
85,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,1
...,...,...
6552,CO[C@H]1C=CO[C@@]2(C)Oc3c(C)c(O)c4c(c3C2=O)C2=...,0
6556,CC(C)=CCC/C(C)=C/C=C\C(C)=C\C=C\C(C)=C\C=C/C=C...,1
6578,CC(=O)N[C@H]1[C@@H](O[C@@H]2C(C(=O)[O-])C[C@@H...,0
6580,CC(C)(CO)[C@@H](O)C(=O)NCCC(=O)[O-].CC(C)(CO)[...,1


In [30]:
datasets[1].to_csv('/Users/james/Documents/Honours/Data/Misc/NR-AR-LBD_target_test.csv')

In [31]:
import pandas as pd
import numpy as np
#Data import
dataset = pd.read_csv('/Users/james/Documents/Honours/Data/Misc/NR-AR-LBD_target_test.csv')
smiles = dataset['SMILES'].tolist()
toxvals = dataset['Toxicity_Value'].tolist()
dflist =  list(zip(smiles, toxvals))
df = pd.DataFrame(dflist, columns = ['SMILES', 'Toxicity_Values'])

In [32]:
#Reads in files from a given folder, converting them to a list
from pathlib import Path
path = Path('/Users/james/Documents/Honours/Data/Misc/tempdump/')

count = 0
CHEMBL_ID_LIST = []
for files in path.iterdir():
    with open(files, 'r') as file:
        chemblids = file.readlines()
    idlist = []
    for ids in chemblids:
        idlist.append(ids.strip())
    
    CHEMBL_ID_LIST.append(idlist)

In [35]:
#Creates a list of unique CHEMBLIDs 
sublist = []
for items in CHEMBL_ID_LIST:
    for ids in items:
        sublist.append(ids)

uniqids = set(sublist)
uniqids = list(uniqids)

In [36]:
#Creates a 1 hot encoded list of CHEMBLIDs
def onehotencoder(uniqids, inputlist):
    newlist = []
    for ids in uniqids:
        if ids in inputlist:
            newlist.append(1)
        else:
            newlist.append(0)
    return newlist

idlist = []
for items in CHEMBL_ID_LIST:
    idlist.append(onehotencoder(uniqids, items))
    

In [40]:
len(uniqids)

289

In [37]:
#Converts list into a dataframe and then appends it to existing file
temp = pd.DataFrame(idlist, columns = [uniqids])

targetsdf = pd.concat([df, temp], axis=1)
targetsdf.to_csv('/Users/james/Documents/Honours/Data/Misc/tempdrugtargets.csv', index=False)

In [41]:
targetsdf

,SMILES,Toxicity_Values,"(CHEMBL5979,)","(CHEMBL1889,)","(CHEMBL311,)","(CHEMBL2140,)","(CHEMBL1293267,)","(CHEMBL1829,)","(CHEMBL4072,)","(CHEMBL3085613,)",...,"(CHEMBL4508,)","(CHEMBL2366504,)","(CHEMBL3229,)","(CHEMBL4150,)","(CHEMBL1075269,)","(CHEMBL3430892,)","(CHEMBL3313832,)","(CHEMBL2329,)","(CHEMBL4358,)","(CHEMBL4625,)"
0,CC(C)C[C@H](NC(=O)[C@H](Cc1ccccc1)NC(=O)c1cncc...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1...,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,Clc1ccc(C(Cn2ccnc2)OCc2csc3c(Cl)cccc23)c(Cl)c1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,Nc1cc(-c2ccncc2)c[nH]c1=O,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,CO[C@H]1C=CO[C@@]2(C)Oc3c(C)c(O)c4c(c3C2=O)C2=...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
476,CC(C)=CCC/C(C)=C/C=C\C(C)=C\C=C\C(C)=C\C=C/C=C...,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
477,CC(=O)N[C@H]1[C@@H](O[C@@H]2C(C(=O)[O-])C[C@@H...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
478,CC(C)(CO)[C@@H](O)C(=O)NCCC(=O)[O-].CC(C)(CO)[...,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [47]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math
df = targetsdf
df.drop('SMILES', axis=1)
df = df.dropna()
dfarray = df.to_numpy()

dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                    random_state=seed, stratify = df['Toxicity_Values'])

size = len(dfarray) / 5
splitsize = math.ceil(size)
empty = [[] for _ in range(5)]
splits = []
for list in empty:
    splits.append(list)

loops = 0
currsplit = 0
#Results in a list of 5 lists that each contain 1/5 of the targetdata
for row in dfarray:
    splits[currsplit].append(row)
    if loops == splitsize:
        loops = 0
        currsplit = currsplit + 1
    loops = loops + 1

folds = []
for split in splits:
    temp = pd.DataFrame(split)
    folds.append(temp)

model_list = []
validlist = []
totalmetrics = []
testdat = []
#For loop that uses each fold once for valid/testing and the rest for training
#Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
iteration = 1
print('========================================')
for split in range(0,5):
    #Set creation
    trainlist = [df for i, df in enumerate(folds) if i != split]
    trainset = pd.concat(trainlist, axis=0)
    testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

    ytrain = trainset[0].values
    xtrain = trainset.iloc[:, 1:]

    ytest = testset[0].values
    xtest = testset.iloc[:, 1:]

    yvalid = validset[0].values
    xvalid = validset.iloc[:, 1:]

    #lists of metric values
    mcclist = []
    bestmetrics = 1
    maxval = 0
    for mtry in range(1, 50):
        #using mtry as the adjusted hyperparameter creates a series of random forests
        rf = RandomForestClassifier(n_estimators=mtry, criterion='entropy', max_depth=None, 
                                min_samples_split=2, min_samples_leaf=1, 
                                min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xtest)
        #calculate metric (mcc)
        mcc = metriccalc(preds, ytest)[12]
        mcclist.append(mcc)
        if mcc > maxval:
            maxval = mcc
            bestmetrics = mtry
    #store best model for the given fold and plot the metric vs mcc value
    rf = RandomForestClassifier(n_estimators=bestmetrics, criterion='entropy', max_depth=None, 
                                min_samples_split=2, min_samples_leaf=1, 
                                min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
    model_list.append(rf)
    testdat.append(maxval)
    totalmetrics.append(mcclist)

    #check models onto validation set, printing various metrics
    model= rf.fit(xtrain, ytrain)
    preds = model.predict(xvalid)
    results = metriccalc(preds, yvalid)
    validlist.append(results)
    print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])

    iteration = iteration + 1

    #get x and y values for the final validation set

toxvals = []
fingerprints = []
for row in tempset:
    toxvals.append(row[0])
    fingerprints.append(row[1:])

#calculates consensus of models on each fingerprint in the final validation set
consensuslist = []
predictions = []
for fp in fingerprints:
    consensus = -1
    fp = fp.reshape(1, -1)
    #appends each model's prediction to a list
    predictions = []
    predlist = []
    for model in model_list:
        preds = model.predict(fp)
        predictions.append(preds)
        predlist.append(preds)
    predictions.append(predlist)

    #finds number of 0s in the prediction list
    zercount = 0
    for num in predictions:
        if num == 0:
            zercount = zercount + 1
    #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
    if zercount < 3 :
        consensus = 1
    else:
        consensus = 0
    consensuslist.append(consensus)
#calculate and print metrics
results = metriccalc(consensuslist, toxvals)
print('\nvalidation metrics of:')
print('positives in data', results[0])
print('negatives in data', results[1])
print('net accuracy =', results[6])
print('mcc =',results[12])

for fold 1 test set mcc of 0 valid set mcc of 0
for fold 2 test set mcc of 0 valid set mcc of 0
for fold 3 test set mcc of 0 valid set mcc of 0
for fold 4 test set mcc of 0 valid set mcc of 0
for fold 5 test set mcc of 0 valid set mcc of 0

validation metrics of:
positives in data 0
negatives in data 0
net accuracy = 0
mcc = 0


In [15]:
import pandas as pd
uni = pd.read_csv('/Users/james/Documents/Honours/Data/Unified_SMILES/Unified_SMILES.csv')
smiles = uni['SMILES'].values
test = 'Cn1c(=O)c2c(ncn2C)n(C)c1=O'
for smile in smiles:
    if smile == test:
        print('bing')

bing


In [ ]:
COc1cccc(C2(O)CCCCC2CN(C)C)c1

In [3]:
uni

,Unnamed: 0,SMILES
0,0,S=c1nc[nH]c2nc[nH]c12
1,1,[NH3+][C@@H](CC(=O)N1CCn2c(nnc2C(F)(F)F)C1)Cc1...
2,2,Nc1nnc(-c2cccc(Cl)c2Cl)c(N)n1
3,3,O=C1[C@H](CC[C@H](O)c2ccc(F)cc2)[C@@H](c2ccc(O...
4,4,O=C(c1ccc(OCC[NH+]2CCCCC2)cc1)c1c(-c2ccc(O)cc2...
...,...,...
17768,17768,c1ccc2c(c1)ccc1cc3ccc4ccccc4c3cc12
17769,17769,c1ccc2c3c4c(cc5ccccc5c4cc2c1)[C@@H]1O[C@H]31
17770,17770,CCCCCCOc1ccc(C(=N)N(CCCC)CCCC)c2ccccc12
17771,17771,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(=N)N)NC(=...


In [ ]:
temp = test.drop